# CLOSE AND OPEN EYE CLASSIFIER TRAINING

Once we have extract the eyes image from the original image we can use and train a classifier using a CNN for labeling eyes as open or close, using our dataset.

This first model has 4 convolutional layers, with 16*n kernels of size 3x3, an output layer with a sigmoid activation funcion for binary classification, and Adam optimizer. Also a reduced LR callback has been used every 5 epochs without improvement in the validation accuracy.

This model has 100% accuracy for train dataset, 97% for test dataset, and 95% for another public dataset download from kaggle.

There is a 2nd model with also 4 convolutional layers, but the 1st layer with less kernels applied, and the callback function for reducing LR has been change to 10 epochs without improvement. It has better accuracy for our dataset, but worse for kaggle dataset.

Finally, this models are saved to .h5 files, so they can be load and used in the eye detection code.

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import os
import shutil
import sklearn
import Augmentor as aug
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import time

In [45]:
def brightness_augment(img, factor=0.5): 
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV) #convert to hsv
    hsv = np.array(hsv, dtype=np.float64)
    #hsv[:, :, 2] = hsv[:, :, 2] * (factor + np.random.uniform()) #scale channel V uniformly
    hsv[:, :, 2] = hsv[:, :, 2] * (factor) #scale channel V uniformly
    hsv[:, :, 2][hsv[:, :, 2] > 255] = 255 #reset out of range values
    rgb = cv2.cvtColor(np.array(hsv, dtype=np.uint8), cv2.COLOR_HSV2RGB)
    return rgb

In [10]:
def create_directory(path):
    if os.path.isdir(path) == False:
        try:
            os.makedirs(path)
        except OSError:
            print ("Creation of the directory %s failed" % path)
        else:
            print ("Successfully created the directory %s " % path)

In [81]:
def load_images(root):
    #Method to load all the images and create X data and labels
    data = []
    for category in sorted(os.listdir(root)):
        for file in sorted(os.listdir(os.path.join(root, category))):
            data.append((category, os.path.join(root, category,  file)))

    df = pd.DataFrame(data, columns=['class', 'file_path'])
    df['class'] = LabelEncoder().fit_transform(df['class'].values)
    photos, labels, names = list(), list(), list()
    for index,row in df.iterrows():
        photo = load_img(row['file_path'], target_size=(80, 80))
        photo = img_to_array(photo)
        photo = photo.astype("float32") / 255
        photos.append(photo)
        labels.append(row['class'])
        names.append(row['file_path'])
    X = np.asarray(photos)
    y = np.asarray(labels)
    return X, y, names

In [12]:
def change_size(root):
    directory = os.path.join(root,"rescaled")
    create_directory(directory)
    for file in sorted(os.listdir(root)):
        if os.path.isfile(os.path.join(root,file)):
            if file.split(".")[1] in ["jpg","bmp","tiff"]:
                path = os.path.join(root,file)
                img = cv2.imread(path)
                image = resize(img, (80, 80), preserve_range=True).astype(np.uint8)
                name = os.path.join(directory,file)
                cv2.imwrite(name, image)

In [13]:
class CustomCallbackReduceLr(keras.callbacks.Callback):
    def __init__(self):
        self.val_accuracy = np.Inf
        self.accumulator = 0
    def on_epoch_begin(self, epoch, logs={}):
        if self.accumulator == 5:
            self.model.optimizer.lr = self.model.optimizer.lr*0.5
            self.accumulator = 0
            print("Changing Learning Rate to ", float(keras.backend.get_value(self.model.optimizer.lr)))
    def on_epoch_end(self, epoch, logs={}):
        current = logs["val_accuracy"]
        if (current - self.val_accuracy) < 0.05:
            self.accumulator = self.accumulator + 1
            print("No change on val accuracy, increment accumulator to ", self.accumulator)
        else:
            self.accumulator = 0
        self.val_accuracy = current

In [14]:
class CustomCallbackReduceLr2(keras.callbacks.Callback):
    def __init__(self):
        self.val_accuracy = np.Inf
        self.accumulator = 0
    def on_epoch_begin(self, epoch, logs={}):
        if self.accumulator == 10:
            self.model.optimizer.lr = self.model.optimizer.lr*0.5
            self.accumulator = 0
            print("Changing Learning Rate to ", float(keras.backend.get_value(self.model.optimizer.lr)))
    def on_epoch_end(self, epoch, logs={}):
        current = logs["val_accuracy"]
        if (current - self.val_accuracy) < 0.05:
            self.accumulator = self.accumulator + 1
            print("No change on val accuracy, increment accumulator to ", self.accumulator)
        else:
            self.accumulator = 0
        self.val_accuracy = current

In [62]:
train_open_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\data\train\Open'
test_open_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\data\test\Open'
train_closed_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\data\train\Closed'
test_closed_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\data\test\Closed'

change_size(train_open_root)
change_size(test_open_root)
change_size(train_closed_root)
change_size(test_closed_root)

Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\data\train\Open\rescaled 
Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\data\test\Open\rescaled 
Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\data\train\Closed\rescaled 


In [65]:
train_open_root_rescaled = os.path.join(train_open_root,"rescaled")
test_open_root_rescaled = os.path.join(test_open_root,"rescaled")
train_closed_root_rescaled = os.path.join(train_closed_root,"rescaled")
test_closed_root_rescaled = os.path.join(test_closed_root,"rescaled")

paths = [train_open_root_rescaled,test_open_root_rescaled,train_closed_root_rescaled,test_closed_root_rescaled]

for path in paths:
    length = len(os.listdir(path))
    augmentor_train_pipeline = aug.Pipeline(path)
    augmentor_train_pipeline.rotate(probability=0.8, max_left_rotation=10, max_right_rotation=10)
    augmentor_train_pipeline.skew_tilt(probability=0.75,magnitude = 0.6)
    augmentor_train_pipeline.random_distortion(probability=0.75, grid_width= 50, grid_height = 50, magnitude = 2)
    augmentor_train_pipeline.flip_left_right(probability=0.75)
    augmentor_train_pipeline.zoom(probability=0.75, min_factor=1.1, max_factor=1.3)
    augmentor_train_pipeline.status()
    augmentor_train_pipeline.sample(length*10)

final_train_open_path = r"C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\train\Open"
final_train_closed_path = r"C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\train\Closed"
final_test_open_path = r"C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Open"
final_test_closed_path = r"C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Closed"

create_directory(final_train_open_path)
create_directory(final_train_closed_path)
create_directory(final_test_open_path)
create_directory(final_test_closed_path)

for files in os.listdir(os.path.join(train_open_root_rescaled,"Output")):
    source = os.path.join(os.path.join(train_open_root_rescaled,"Output"),files)
    destination = os.path.join(final_train_open_path,files)
    img = cv2.imread(source)
    name1 = "0" + files
    name2 = "1" + files
    name3 = "2" + files
    name4 = "3" + files
    dest1 = os.path.join(final_train_open_path,name1)
    dest2 = os.path.join(final_train_open_path,name2)
    dest3 = os.path.join(final_train_open_path,name2)
    dest4 = os.path.join(final_train_open_path,name2)
    bright1 = brightness_augment(img, 1.5)
    bright2 = brightness_augment(img, 2)
    dark1 = brightness_augment(img, 0.75)
    dark2 = brightness_augment(img, 0.6)
    cv2.imwrite(dest1,bright1)
    cv2.imwrite(dest2,bright2)
    cv2.imwrite(dest3,dark1)
    cv2.imwrite(dest4,dark2)
    shutil.move(source, destination) 
shutil.rmtree(train_open_root_rescaled)

for files in os.listdir(os.path.join(test_open_root_rescaled,"Output")):
    source = os.path.join(os.path.join(test_open_root_rescaled,"Output"),files)
    destination = os.path.join(final_test_open_path,files)
    img = cv2.imread(source)
    name1 = "0" + files
    name2 = "1" + files
    name3 = "2" + files
    name4 = "3" + files
    dest1 = os.path.join(final_test_open_path,name1)
    dest2 = os.path.join(final_test_open_path,name2)
    dest3 = os.path.join(final_test_open_path,name2)
    dest4 = os.path.join(final_test_open_path,name2)
    bright1 = brightness_augment(img, 1.5)
    bright2 = brightness_augment(img, 2)
    dark1 = brightness_augment(img, 0.75)
    dark2 = brightness_augment(img, 0.6)
    cv2.imwrite(dest1,bright1)
    cv2.imwrite(dest2,bright2)
    cv2.imwrite(dest3,dark1)
    cv2.imwrite(dest4,dark2)
    shutil.move(source, destination) 
shutil.rmtree(test_open_root_rescaled)

for files in os.listdir(os.path.join(train_closed_root_rescaled,"Output")):
    source = os.path.join(os.path.join(train_closed_root_rescaled,"Output"),files)
    destination = os.path.join(final_train_closed_path,files)
    img = cv2.imread(source)
    name1 = "0" + files
    name2 = "1" + files
    name3 = "2" + files
    name4 = "3" + files
    dest1 = os.path.join(final_train_closed_path,name1)
    dest2 = os.path.join(final_train_closed_path,name2)
    dest3 = os.path.join(final_train_closed_path,name2)
    dest4 = os.path.join(final_train_closed_path,name2)
    bright1 = brightness_augment(img, 1.5)
    bright2 = brightness_augment(img, 2)
    dark1 = brightness_augment(img, 0.75)
    dark2 = brightness_augment(img, 0.6)
    cv2.imwrite(dest1,bright1)
    cv2.imwrite(dest2,bright2)
    cv2.imwrite(dest3,dark1)
    cv2.imwrite(dest4,dark2)
    shutil.move(source, destination) 
shutil.rmtree(train_closed_root_rescaled)
    
for files in os.listdir(os.path.join(test_closed_root_rescaled,"Output")):
    source = os.path.join(os.path.join(test_closed_root_rescaled,"Output"),files)
    destination = os.path.join(final_test_closed_path,files)
    img = cv2.imread(source)
    name1 = "0" + files
    name2 = "1" + files
    name3 = "2" + files
    name4 = "3" + files
    dest1 = os.path.join(final_test_closed_path,name1)
    dest2 = os.path.join(final_test_closed_path,name2)
    dest3 = os.path.join(final_test_closed_path,name2)
    dest4 = os.path.join(final_test_closed_path,name2)
    bright1 = brightness_augment(img, 1.5)
    bright2 = brightness_augment(img, 2)
    dark1 = brightness_augment(img, 0.75)
    dark2 = brightness_augment(img, 0.6)
    cv2.imwrite(dest1,bright1)
    cv2.imwrite(dest2,bright2)
    cv2.imwrite(dest3,dark1)
    cv2.imwrite(dest4,dark2)
    shutil.move(source, destination) 
shutil.rmtree(test_closed_root_rescaled)

Initialised with 627 image(s) found.
Output directory set to C:\Users\guill\Desktop\INDIZEN\Capstone\data\train\Open\rescaled\output.Operations: 5
	0: RotateRange (probability=0.8 max_left_rotation=-10 max_right_rotation=10 )
	1: Skew (probability=0.75 skew_type=TILT magnitude=0.6 )
	2: Distort (probability=0.75 grid_width=50 grid_height=50 magnitude=2 randomise_magnitude=True )
	3: Flip (probability=0.75 top_bottom_left_right=LEFT_RIGHT )
	4: Zoom (probability=0.75 min_factor=1.1 max_factor=1.3 )
Images: 627
Classes: 1
	Class index: 0 Class label: rescaled 
Dimensions: 1
	Width: 80 Height: 80
Formats: 1
	 JPEG

You can remove operations using the appropriate index and the remove_operation(index) function.


Processing <PIL.Image.Image image mode=RGB size=80x80 at 0x1EB0ACC93A0>: 100%|█| 6270/6270 [04:07<00:00, 25.33 Samples/


Initialised with 117 image(s) found.
Output directory set to C:\Users\guill\Desktop\INDIZEN\Capstone\data\test\Open\rescaled\output.Operations: 5
	0: RotateRange (probability=0.8 max_left_rotation=-10 max_right_rotation=10 )
	1: Skew (probability=0.75 skew_type=TILT magnitude=0.6 )
	2: Distort (probability=0.75 grid_width=50 grid_height=50 magnitude=2 randomise_magnitude=True )
	3: Flip (probability=0.75 top_bottom_left_right=LEFT_RIGHT )
	4: Zoom (probability=0.75 min_factor=1.1 max_factor=1.3 )
Images: 117
Classes: 1
	Class index: 0 Class label: rescaled 
Dimensions: 1
	Width: 80 Height: 80
Formats: 1
	 JPEG

You can remove operations using the appropriate index and the remove_operation(index) function.


Processing <PIL.Image.Image image mode=RGB size=80x80 at 0x1EB0BAC85B0>: 100%|█| 1170/1170 [00:44<00:00, 26.51 Samples/


Initialised with 627 image(s) found.
Output directory set to C:\Users\guill\Desktop\INDIZEN\Capstone\data\train\Closed\rescaled\output.Operations: 5
	0: RotateRange (probability=0.8 max_left_rotation=-10 max_right_rotation=10 )
	1: Skew (probability=0.75 skew_type=TILT magnitude=0.6 )
	2: Distort (probability=0.75 grid_width=50 grid_height=50 magnitude=2 randomise_magnitude=True )
	3: Flip (probability=0.75 top_bottom_left_right=LEFT_RIGHT )
	4: Zoom (probability=0.75 min_factor=1.1 max_factor=1.3 )
Images: 627
Classes: 1
	Class index: 0 Class label: rescaled 
Dimensions: 1
	Width: 80 Height: 80
Formats: 1
	 JPEG

You can remove operations using the appropriate index and the remove_operation(index) function.


Processing <PIL.Image.Image image mode=RGB size=80x80 at 0x1EBA91DEDF0>: 100%|█| 6270/6270 [04:14<00:00, 24.64 Samples/


Initialised with 116 image(s) found.
Output directory set to C:\Users\guill\Desktop\INDIZEN\Capstone\data\test\Closed\rescaled\output.Operations: 5
	0: RotateRange (probability=0.8 max_left_rotation=-10 max_right_rotation=10 )
	1: Skew (probability=0.75 skew_type=TILT magnitude=0.6 )
	2: Distort (probability=0.75 grid_width=50 grid_height=50 magnitude=2 randomise_magnitude=True )
	3: Flip (probability=0.75 top_bottom_left_right=LEFT_RIGHT )
	4: Zoom (probability=0.75 min_factor=1.1 max_factor=1.3 )
Images: 116
Classes: 1
	Class index: 0 Class label: rescaled 
Dimensions: 1
	Width: 80 Height: 80
Formats: 1
	 JPEG

You can remove operations using the appropriate index and the remove_operation(index) function.


Processing <PIL.Image.Image image mode=RGB size=80x80 at 0x1EB09C35D30>: 100%|█| 1160/1160 [00:53<00:00, 21.79 Samples/


Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\train\Open 
Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\train\Closed 
Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Open 
Successfully created the directory C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Closed 


In [86]:
train_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\train'
test_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test'
x_train, y_train, train_names = load_images(train_root)
x_test, y_test, test_names = load_images(test_root)

In [71]:
keras.backend.clear_session()
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu',padding='same', input_shape=(80, 80, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 80, 80, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 128)       7

In [72]:
change_lr_callback = CustomCallbackReduceLr2()
history = model.fit(x_train, y_train, batch_size=30, epochs=100, verbose=1, validation_split=0.1, callbacks = [change_lr_callback])

Epoch 1/100
1129/1129 [==============================] - 101s 89ms/step - loss: 0.1259 - accuracy: 0.9531 - val_loss: 0.0314 - val_accuracy: 0.9854
No change on val accuracy, increment accumulator to  1
Epoch 2/100
1129/1129 [==============================] - 102s 90ms/step - loss: 0.0337 - accuracy: 0.9886 - val_loss: 0.0063 - val_accuracy: 0.9984
No change on val accuracy, increment accumulator to  2
Epoch 3/100
1129/1129 [==============================] - 107s 94ms/step - loss: 0.0213 - accuracy: 0.9923 - val_loss: 0.0049 - val_accuracy: 0.9981
No change on val accuracy, increment accumulator to  3
Epoch 4/100
1129/1129 [==============================] - 108s 95ms/step - loss: 0.0141 - accuracy: 0.9957 - val_loss: 0.0012 - val_accuracy: 1.0000
No change on val accuracy, increment accumulator to  4
Epoch 5/100
1129/1129 [==============================] - 106s 94ms/step - loss: 0.0121 - accuracy: 0.9961 - val_loss: 0.0024 - val_accuracy: 0.9992
No change on val accuracy, increment acc

No change on val accuracy, increment accumulator to  6
Epoch 77/100
1129/1129 [==============================] - 98s 87ms/step - loss: 9.2443e-07 - accuracy: 1.0000 - val_loss: 2.6440e-12 - val_accuracy: 1.0000
No change on val accuracy, increment accumulator to  7
Epoch 78/100
1129/1129 [==============================] - 97s 86ms/step - loss: 5.8584e-08 - accuracy: 1.0000 - val_loss: 2.6465e-12 - val_accuracy: 1.0000
No change on val accuracy, increment accumulator to  8
Epoch 79/100
1129/1129 [==============================] - 97s 86ms/step - loss: 1.1406e-07 - accuracy: 1.0000 - val_loss: 2.5767e-12 - val_accuracy: 1.0000
No change on val accuracy, increment accumulator to  9
Epoch 80/100
1129/1129 [==============================] - 97s 86ms/step - loss: 1.6378e-07 - accuracy: 1.0000 - val_loss: 2.4349e-12 - val_accuracy: 1.0000
No change on val accuracy, increment accumulator to  10
Epoch 81/100
Changing Learning Rate to  3.906250185536919e-06
1129/1129 [===========================

In [91]:
y_predicted = []
start_time = time.time()
for i in range(len(y_test)):
    out_probabilities = model.predict(x_test[i:i+1])
    number_detected = 1 if out_probabilities[0][0] > 0.5 else 0
    y_predicted.append(number_detected)
    if number_detected != y_test[i]:
        print(test_names[i])
        print(out_probabilities[0][0])
end_time = time.time()
mean = (end_time - start_time) / len(y_test)
a = model.evaluate(x_test, y_test)
print(classification_report(y_test, y_predicted))
print(confusion_matrix(y_test, y_predicted))
print("Time for prediction: {}".format(mean))

C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Closed\0rescaled_original__137.jpg_46a66481-b000-470b-9d60-3b2898dc99fb.jpg
0.5315014
C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Closed\0rescaled_original__148.jpg_b8567930-264d-4bbf-af8b-87b583dc8d12.jpg
0.86216235
C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Closed\0rescaled_original__163.jpg_3f8cf67b-a2d6-4b39-8cc2-684815b717ea.jpg
0.9915544
C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Closed\0rescaled_original__168.jpg_395ab72d-d6d0-406a-b4c2-c2601ef49db7.jpg
1.0
C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Closed\0rescaled_original__168.jpg_45113875-8024-40bd-aa55-e22b7a47dbd5.jpg
0.9435326
C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Closed\0rescaled_original__383.jpg_5e7266f0-f70d-471d-bf0f-19fd6c792bfd.jpg
0.9384149
C:\Users\guill\Desktop\INDIZEN\Capstone\final_dataset\test\Closed\0rescaled_original__383.jpg_a838bae7-d510-4a3a-aa94-f331101d6de5.jpg


In [74]:
y_predicted = []
start_time = time.time()
for i in range(len(y_train)):
    out_probabilities = model.predict(x_train[i:i+1])
    number_detected = 1 if out_probabilities[0][0] > 0.5 else 0
    y_predicted.append(number_detected)
    if number_detected != y_train[i]:
        print(train_names[i])
end_time = time.time()
mean = (end_time - start_time) / len(y_train)
a = model.evaluate(x_train, y_train)
print(classification_report(y_train, y_predicted))
print(confusion_matrix(y_train, y_predicted))
print("Time for prediction: {}".format(mean))

1176/1176 [==============================] - 31s 26ms/step - loss: 6.7375e-09 - accuracy: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18810
           1       1.00      1.00      1.00     18810

    accuracy                           1.00     37620
   macro avg       1.00      1.00      1.00     37620
weighted avg       1.00      1.00      1.00     37620

[[18810     0]
 [    0 18810]]
Time for prediction: 0.0371910613246068


In [88]:
pruebas_root = r'C:\Users\guill\Desktop\INDIZEN\Capstone\yolov5-face-master\eye classifier notebooks\pruebas'
x_pruebas, y_pruebas, pruebas_names = load_images(pruebas_root)

In [90]:
y_predicted = []
start_time = time.time()
for i in range(len(y_pruebas)):
    out_probabilities = model.predict(x_pruebas[i:i+1])
    number_detected = 1 if out_probabilities[0][0] > 0.5 else 0
    y_predicted.append(number_detected)
    if number_detected != y_pruebas[i]:
        print(pruebas_names[i])
        print(out_probabilities[0][0])
end_time = time.time()
mean = (end_time - start_time) / len(y_pruebas)
a = model.evaluate(x_pruebas, y_pruebas)
print(classification_report(y_pruebas, y_predicted))
print(confusion_matrix(y_pruebas, y_predicted))
print("Time for prediction: {}".format(mean))

C:\Users\guill\Desktop\INDIZEN\Capstone\yolov5-face-master\eye classifier notebooks\pruebas\Closed\s0006_00016_0_1_0_0_0_01.png
0.9999964
C:\Users\guill\Desktop\INDIZEN\Capstone\yolov5-face-master\eye classifier notebooks\pruebas\Closed\s0006_00334_0_1_0_1_0_01.png
1.0
C:\Users\guill\Desktop\INDIZEN\Capstone\yolov5-face-master\eye classifier notebooks\pruebas\Closed\s0006_00336_0_1_0_1_0_01.png
1.0
C:\Users\guill\Desktop\INDIZEN\Capstone\yolov5-face-master\eye classifier notebooks\pruebas\Closed\s0006_00337_0_1_0_1_0_01.png
1.0
C:\Users\guill\Desktop\INDIZEN\Capstone\yolov5-face-master\eye classifier notebooks\pruebas\Closed\s0006_00338_0_1_0_1_0_01.png
1.0
C:\Users\guill\Desktop\INDIZEN\Capstone\yolov5-face-master\eye classifier notebooks\pruebas\Closed\s0006_00340_0_1_0_1_0_01.png
1.0
C:\Users\guill\Desktop\INDIZEN\Capstone\yolov5-face-master\eye classifier notebooks\pruebas\Closed\s0006_00342_0_1_0_1_0_01.png
1.0
C:\Users\guill\Desktop\INDIZEN\Capstone\yolov5-face-master\eye classif

In [84]:
root2 = r'C:\Users\guill\Desktop\INDIZEN\Capstone\YOLOV5\video'
x_prueba2, y_prueba2, prueba2_names = load_images(root2)
y_predicted = []
for i in range(len(y_prueba2)):
    out_probabilities = model.predict(x_prueba2[i:i+1])
    number_detected = 1 if out_probabilities[0][0] > 0.05 else 0
    y_predicted.append(number_detected)
    if number_detected != y_prueba2[i]:
        print(prueba2_names[i])
a = model.evaluate(x_prueba2, y_prueba2)
print(classification_report(y_prueba2, y_predicted))
print(confusion_matrix(y_prueba2, y_predicted))

C:\Users\guill\Desktop\INDIZEN\Capstone\YOLOV5\video\Closed\eye_mauricio305.jpg
22/22 [==============================] - 0s 19ms/step - loss: 0.0179 - accuracy: 0.9986
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       326
           1       1.00      1.00      1.00       371

    accuracy                           1.00       697
   macro avg       1.00      1.00      1.00       697
weighted avg       1.00      1.00      1.00       697

[[325   1]
 [  0 371]]


In [80]:
model.save("model/eye_classifier1_v3.h5")
#model2.save("model/eye_classifier2_v2.h5")
print("Saved model to disk")

Saved model to disk
